In [12]:
#load package
import sys
sys.path.append('../')

import numpy as np
import PIL.Image
import re

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from os import listdir
import matplotlib.pyplot as plt
from sklearn.metrics import pairwise_distances

import pretrained_networks
from training.misc import load_pkl
import tensorflow as tf
from training import dataset
from training import misc


import random
import cv2
import dlib
import bz2
import math
from sklearn import svm
from sklearn import manifold, datasets

from scipy.spatial import distance

from training.misc import load_pkl 
from scipy.spatial import distance

index = 0

In [13]:

network_pkl = 'network-snapshot_4.pkl'
M, reM, D, G, C, Gs = misc.load_pkl(network_pkl)


ModuleNotFoundError: No module named 'tensorflow.contrib'

In [ ]:
# open session
sess = tf.get_default_session()

In [ ]:
def generate_from_img_file_reduce(Gs, num=1, index=0, label=0):
    num_layers = Gs.components.synthesis.input_shape[1]
    w = W_c[label][index:index+num]
    label2 = Label_c[label][index:index+num]
    index = Index_c[label][index:index+num]
    w_ = np.tile(w, (1, num_layers, 1))
    img = Gs.components.synthesis.run(w_, **Gs_kwargs)
    return w, img, label2, index

In [ ]:
#load image's latent code 
if True:
    W_latent_ok = np.load('W_latent_ok4_all_clear.npy')
    Label_latent_ok = np.load('Label_latent_ok4_all_clear.npy')
    Index_latent_ok = np.load('Index_latent_ok4_all_clear.npy')
    W_c = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    P_c = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    Label_c = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    Index_c = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    for i in range(len(W_latent_ok)):
        label = Label_latent_ok[i]
        c = label[0]+2*label[1]+4*label[2]+8*label[3]
        W_c[c].append(W_latent_ok[i,0,:])
        p, s = M.run(W_latent_ok[i,0,:], phase=False)
        P_c[c].append(p)
        Label_c[c].append(Label_latent_ok[i])
        Index_c[c].append(Index_latent_ok[i])
        print(i,end = '\r',flush=True)
    for i in range(16):
        W_c[i] = np.array(W_c[i])
        P_c[i] = np.array(P_c[i]) 
        Label_c[i] = np.array(Label_c[i])  
        Index_c[i] = np.array(Index_c[i])  
    W_c = np.array(W_c)
    P_c = np.array(P_c)
    Label_c = np.array(Label_c)
    Index_c = np.array(Index_c)

In [ ]:
def generate_add_allnoise_from_w_batch(M, reM, Gs, w, sess, epsilon=1.0,sens=1.0,lipchitz=1.0, batch_size=1, m_type='G'):
    num_layers = Gs.components.synthesis.input_shape[1]

    img1 = Gs.components.synthesis.run(w, **Gs_kwargs)

    p, s = M.run(np.array([w[0][0]]), phase=False)
    p /= np.linalg.norm(p)
    p_n = []
    
    for i in range(batch_size):
        p_refind = clip_our(np.array([w[0][0]]), _addnoise(np.array([w[0][0]])),sens)
        p_n.append(_addnoise_p(p_refind,epsilon,sens)[0])
    p_n = np.array(p_n)
    
    w,_ = reM.run(p, phase=False)
    w_ = np.tile(w[:, np.newaxis], [1, num_layers, 1])
    #p_d = np.linalg.norm(n)
    #p_n = p_n
    w_n,_ = reM.run(p_n, phase=False)
    w_d = np.linalg.norm(w_n-w)
    w_n = np.tile(w_n[:, np.newaxis], [1, num_layers, 1])
    img3 = Gs.components.synthesis.run(w_, **Gs_kwargs)
    img4 = Gs.components.synthesis.run(w_n, **Gs_kwargs)
    return img1, img3, img4, p_n #(original image, remap image, dp-image, dp-z-space) 
def get_noise_image(delta=1e-4, epsilon=4.0, index=0, label_index=0):
    #filename = './evaluate_data/'
    recog_rate = 0
    recog_rate2 = 0
    count_total = 0
    
    #total = 100
    # test mapping and add noise result

    w, img1, label, index_c = generate_from_img_file_reduce(Gs, index=index, label=label_index)
    filename = '../datasets/celeba-128hq/'+str(t_n_c[index_c][0])
    o_img = cv2.imread(filename)
    #data_index.append(index_c)
    label = label[0]

    count = 5
    IMG, _, IMG_DP, _ = generate_add_allnoise_from_w_batch(M, reM, Gs, w, sess, epsilon=epsilon,sens=distance_p[label_index],m_type='L',batch_size=count)
      
    return IMG, IMG_DP

In [ ]:
org, dpimage = get_noise_image(delta=1e-4, epsilon=0.1, label_index=label, index=index)

%matplotlib inline
cv2.imshow("org image", org[0])
for i in range(5):
    cv2.imshow("dp-image_"+str(i), dpimage[i])
cv2.waitKey(0)